<a href="https://colab.research.google.com/github/Electromensch/nlp_need/blob/main/Scrapping_pandas_formation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.5
!pip install pandas==1.3

In [ ]:
import nltk
from bs4 import BeautifulSoup
import requests
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
url_kor = 'https://musulmanin.com/koran-na-russkom.html'
url_base = 'https://quran-online.ru/'
html_doc = requests.get(url_base).text
soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
all_links2 = [re.findall(r'https://quran-online.ru/\d+/saadi', link.get('href')) for link in soup.find_all('a')]
all_links2 = [link[0] for link in all_links2 if link]

In [ ]:
mylinks = list(dict.fromkeys(all_links2))

In [ ]:
chunks = []
for link in range(len(mylinks)):
    html_doc2 = requests.get(mylinks[link]).text
    soup2 = BeautifulSoup(html_doc2, 'html.parser')
    need_text = soup2.find_all("div", attrs={'class': "ayat-text-type russian active", 'style': "display:block"})
    chunks += [i.getText().strip().replace('\n', ' ').replace(r'[!:.;?]\.', '.') for i in need_text]

In [ ]:
koran = ' '.join(chunks)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ebooklib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 28.5 MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-py3-none-any.whl size=38184 sha256=2d54f77da20d16dda6ca0d8e172ed0bc126468f1dfc3848df09a4738bdc224ba
  Stored in directory: /root/.cache/pip/wheels/43/39/fd/db4f652431a55d28472ba7f5f7c9a8efad03b97f443a48ea2f
Successfully built ebooklib


In [ ]:
import ebooklib
from ebooklib import epub

In [ ]:
!rm -r nlp_need
!git clone https://github.com/Electromensch/nlp_need

rm: cannot remove 'nlp_need': No such file or directory
Cloning into 'nlp_need'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [ ]:
book = epub.read_epub('nlp_need/restruct.epub')
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))

In [ ]:
texts = []

for c in items:
    soup3 = BeautifulSoup(c.get_body_content(), 'html.parser')
    text = [para.get_text() for para in soup3.find_all('p')]
    texts += text
    
tesak = ' '.join(texts)
tesak=tesak.replace('\\n', '')

Data Frame creation

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tesak_sent = sent_tokenize(tesak, language="russian")
koran_sent = sent_tokenize(koran, language="russian")

In [ ]:
df1 = pd.DataFrame(tesak_sent, columns=['sentences'])
df1['text'] = 'Tesak'
df1.head()
df2 = pd.DataFrame(koran_sent, columns=['sentences'])
df2['text'] = 'Koran'
df2.head()

,sentences,text
0,"Во имя Аллаха, Милостивого, Милосердного!",Koran
1,Это означает: я начинаю во имя Аллаха.,Koran
2,Из лексического анализа этого словосочетания я...,Koran
3,"Аллах - одно из этих имен, означающее «Бог, Ко...",Koran
4,Прекрасные имена Милостивый и Милосердный свид...,Koran


In [ ]:
df = pd.concat([df1, df2])
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

In [ ]:
df.head()

,sentences,text
0,"Вот принесли эти задания в колледж, надо решит...",Tesak
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak
3,"Только я сказал, сразу меня начали просить по...",Tesak
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak


In [ ]:
df.to_pickle('nlp_pickle')

In [ ]:
from nltk.probability import FreqDist
import operator

In [ ]:
import spacy

In [ ]:
!python3 -m spacy download ru_core_news_md

In [ ]:
sp = spacy.load("ru_core_news_md")

In [ ]:
df['tokens'] = df['sentences'].apply(lambda x: sp(x))

In [ ]:
df.head()

,sentences,text,tokens,pos_tags
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"(Вот, принесли, эти, задания, в, колледж, ,, н...","{'вот': 'PART', 'принести': 'VERB', 'этот': 'D..."
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"(Когда, -, то, у, меня, по, химии, была, пятер...","{'когда': 'ADV', '-': 'ADV', 'то': 'ADV', 'у':..."
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"(На, зоне, не, нужно, больше, знать, и, уметь,...","{'на': 'ADP', 'зона': 'NOUN', 'не': 'PART', 'н..."
3,"Только я сказал, сразу меня начали просить по...",Tesak,"(Только, , я, сказал, ,, сразу, меня, начали,...","{'только': 'PART', ' ': 'SPACE', 'я': 'PRON', ..."
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"(Мы, садимся, ,, я, открываю, эти, задачи, и, ...","{'мы': 'PRON', 'садиться': 'VERB', ',': 'PUNCT..."


In [ ]:
df['pos_tags'] = df['tokens'].apply(lambda x: {i.lemma_: i.pos_ for i in x})

In [ ]:
df['ner'] = df['tokens'].apply(lambda x: )

In [ ]:
df_train = df.head()

In [ ]:
df_train['ner'] = df_train['tokens'].apply(lambda x: x.ents)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_train.head()

,sentences,text,tokens,pos_tags,ner
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"(Вот, принесли, эти, задания, в, колледж, ,, н...","{'вот': 'PART', 'принести': 'VERB', 'этот': 'D...",()
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"(Когда, -, то, у, меня, по, химии, была, пятер...","{'когда': 'ADV', '-': 'ADV', 'то': 'ADV', 'у':...",()
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"(На, зоне, не, нужно, больше, знать, и, уметь,...","{'на': 'ADP', 'зона': 'NOUN', 'не': 'PART', 'н...",()
3,"Только я сказал, сразу меня начали просить по...",Tesak,"(Только, , я, сказал, ,, сразу, меня, начали,...","{'только': 'PART', ' ': 'SPACE', 'я': 'PRON', ...",()
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"(Мы, садимся, ,, я, открываю, эти, задачи, и, ...","{'мы': 'PRON', 'садиться': 'VERB', ',': 'PUNCT...",()


In [ ]:
df = pd.read_pickle('nlp_pickle')

In [ ]:
!pip install -q deeppavlov

In [ ]:
!python -m deeppavlov install ner_rus
!python -m deeppavlov install ner_rus_bert

In [ ]:
from deeppavlov import configs, build_model

config_path = configs.ner.ner_rus


In [ ]:
from deeppavlov.models.tokenizers.nltk_tokenizer import NLTKTokenizer

In [ ]:
toknz = NLTKTokenizer(tokenizer='wordpunct_tokenize')

In [ ]:
ner = build_model(config_path, download=True)

In [ ]:
def for_bert(text):
  tokens, tags = ner([text])
  return [(tok, tag) for tok, tag in zip(tokens[0], tags[0])]  

In [ ]:
df_train['ner'] = df_train['sentences'].apply(for_bert)

In [ ]:
df_train.ner

0    {'Вот': 'O', 'принесли': 'O', 'эти': 'O', 'зад...
1    {'Когда': 'O', '-': 'O', 'то': 'O', 'у': 'O', ...
2    {'На': 'O', 'зоне': 'O', 'не': 'O', 'нужно': '...
3    {'Только': 'O', 'я': 'O', 'сказал': 'O', ',': ...
4    {'Мы': 'O', 'садимся': 'O', ',': 'O', 'я': 'O'...
Name: ner, dtype: object

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
tqdm.pandas(desc="Example Desc")

In [ ]:
df['ner'] = df['sentences'].progress_apply(for_bert)

Example Desc: 100%|██████████| 65764/65764 [11:47<00:00, 92.98it/s]


In [ ]:
df_train = df.head()

In [ ]:
df_train['tokens'] = df_train.sentences.progress_apply(lambda x: toknz(x))

Example Desc: 100%|██████████| 5/5 [00:00<00:00, 350.20it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  If the values are not callable, (e.g. a Series, scalar, or array),


In [ ]:
from nltk.tokenize import word_tokenize#text, language='english')

In [ ]:
df_train['tokens'] = df_train.sentences.progress_apply(lambda x: word_tokenize(x, language='russian'))

In [ ]:
df['tokens'] = df.sentences.progress_apply(lambda x: word_tokenize(x, language='russian'))

Example Desc: 100%|██████████| 65764/65764 [00:11<00:00, 5508.03it/s]


In [ ]:
df_train

,sentences,text,ner,tokens
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"[(Вот, O), (принесли, O), (эти, O), (задания, ...","[Вот, принесли, эти, задания, в, колледж, ,, н..."
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"[(Когда-то, O), (у, O), (меня, O), (по, O), (х...","[Когда-то, у, меня, по, химии, была, пятерка, ..."
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"[(На, O), (зоне, O), (не, O), (нужно, O), (бол...","[На, зоне, не, нужно, больше, знать, и, уметь,..."
3,"Только я сказал, сразу меня начали просить по...",Tesak,"[(Только, O), (я, O), (сказал, O), (,, O), (ср...","[Только, я, сказал, ,, сразу, меня, начали, пр..."
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"[(Мы, O), (садимся, O), (,, O), (я, O), (откры...","[Мы, садимся, ,, я, открываю, эти, задачи, и, ..."


In [ ]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

In [ ]:
import pymorphy2

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def morph_tag(x):
  return [(i, str(morph.parse(i)[0].tag.POS)) for i in x]

In [ ]:
df_train['tags'] = df_train.tokens.progress_apply(morph_tag)

In [ ]:
df['tags'] = df.tokens.progress_apply(morph_tag)

Example Desc: 100%|██████████| 65764/65764 [05:16<00:00, 207.86it/s]


In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/mypickle2')

In [ ]:
df.head()

,sentences,text,ner,tokens,tags,ner_new,ner_final
0,"Вот принесли эти задания в колледж, надо решит...",Tesak,"[(Вот, O), (принесли, O), (эти, O), (задания, ...","[Вот, принесли, эти, задания, в, колледж, ,, н...","[(Вот, PRCL), (принесли, VERB), (эти, ADJF), (...","[(Вот, O), (принесли, O), (эти, O), (задания, ...",[]
1,"Когда-то у меня по химии была пятерка, а я по ...",Tesak,"[(Когда-то, O), (у, O), (меня, O), (по, O), (х...","[Когда-то, у, меня, по, химии, была, пятерка, ...","[(Когда-то, ADVB), (у, PREP), (меня, NPRO), (п...","[(Когда-то, O), (у, O), (меня, O), (по, O), (х...",[]
2,"На зоне не нужно больше знать и уметь, кроме с...",Tesak,"[(На, O), (зоне, O), (не, O), (нужно, O), (бол...","[На, зоне, не, нужно, больше, знать, и, уметь,...","[(На, PREP), (зоне, NOUN), (не, PRCL), (нужно,...","[(На, O), (зоне, O), (не, O), (нужно, O), (бол...",[]
3,"Только я сказал, сразу меня начали просить по...",Tesak,"[(Только, O), (я, O), (сказал, O), (,, O), (ср...","[Только, я, сказал, ,, сразу, меня, начали, пр...","[(Только, PRCL), (я, NPRO), (сказал, VERB), (,...","[(Только, O), (я, O), (сказал, O), (,, O), (ср...",[]
4,"Мы садимся, я открываю эти задачи и понимаю, ч...",Tesak,"[(Мы, O), (садимся, O), (,, O), (я, O), (откры...","[Мы, садимся, ,, я, открываю, эти, задачи, и, ...","[(Мы, NPRO), (садимся, VERB), (,, None), (я, N...","[(Мы, O), (садимся, O), (,, O), (я, O), (откры...",[]


In [ ]:
def shiterator(d):
  my_lst = []
  for i in d:
    
    if i[1] != 'O':
      my_lst.append(i)
  return my_lst

In [ ]:
def shiterator2(d):
  my_lst = []
  new_key = ''
  for i in d:
    if i[1] == 'B-PER':
      new_key = i[0]
      my_lst.append((i[0], 'PER'))
    elif i[1] == 'I-PER':
      if new_key != '':
        final_key = new_key + ' ' + i[0]
        final_value = 'PER'
        my_lst.pop(-1)
        my_lst.append((final_key, final_value))
        new_key = ''
      else:
        new_key = ''
        my_lst.append((i[0], 'PER'))
    else:
      new_key = ''
      my_lst.append((i[0], i[1]))
  return my_lst

In [ ]:
def shiterator3(d):
  my_lst = []
  new_key = ''
  for i in d:
    if i[1] == 'B-LOC':
      new_key = i[0]
      my_lst.append((i[0], 'LOC'))
    elif i[1] == 'I-LOC':
      if new_key != '':
        final_key = new_key + ' ' + i[0]
        final_value = 'LOC'
        my_lst.pop(-1)
        my_lst.append((final_key, final_value))
        new_key = ''
      else:
        new_key = ''
        my_lst.append((i[0], 'LOC'))
    else:
      new_key = ''
      my_lst.append((i[0], i[1]))
  return my_lst

In [ ]:
def shiterator4(d):
  my_lst = []
  new_key = ''
  for i in d:
    if i[1] == 'B-ORG':
      new_key = i[0]
      my_lst.append((i[0], 'ORG'))
    elif i[1] == 'I-ORG':
      if new_key != '':
        final_key = new_key + ' ' + i[0]
        final_value = 'ORG'
        my_lst.pop(-1)
        my_lst.append((final_key, final_value))
        new_key = ''
      else:
        new_key = ''
        my_lst.append((i[0], 'ORG'))
    else:
      new_key = ''
      my_lst.append((i[0], i[1]))
  return my_lst

In [ ]:
df['ner_new'] = df.ner_new.progress_apply(shiterator4)

Example Desc: 100%|██████████| 65764/65764 [00:02<00:00, 32312.83it/s]


In [ ]:
df['ner_final'] = df.ner_new.progress_apply(shiterator)

Example Desc: 100%|██████████| 65764/65764 [00:00<00:00, 170227.57it/s]


In [ ]:
df.to_pickle('mypickle3')